In [47]:
import polars as pl
import polars.selectors as cs

Part 1

In [ ]:
# from io import StringIO
# IN_ = """123 328  51 64 
#  45 64  387 23 
#   6 98  215 314
# *   +   *   +  """
# IN = StringIO(IN_)
IN = "d6_in.txt"
(
  pl.read_csv(IN, has_header=False)
  .with_columns(
    pl.col("column_1")
    .str
    .strip_chars(" ")
    .str
    .replace_all(r"\s+", ",")
    .str
    .split(",")
    .list
    .to_struct(),
  )
  .unnest("column_1")
  .reverse()
  .transpose()
  .rename({"column_0": "ops"})
  .with_columns(cs.contains("column_").cast(pl.Int64))
  .with_columns(
    total=
    pl.when(pl.col("ops") == "+")
    .then(pl.reduce(function=lambda acc, x: acc + x, exprs=cs.contains("column_")))
    .otherwise(pl.reduce(function=lambda acc, x: acc * x, exprs=cs.contains("column_")))
  )
  .select(pl.col("total").sum())
)

total
i64
5877594983578


Part 2

In [141]:
# 123 328  51 64 
#  45 64  387 23 
#   6 98  215 314
# *   +   *   +
from io import StringIO
IN_ = """123 328  51 64 
 45 64  387 23 
  6 98  215 314
*   +   *   +  """
IN = StringIO(IN_)
# IN = "d6_in.txt"
pl.Config(set_fmt_table_cell_list_len=20)
pl.Config(set_tbl_width_chars=50)
print(
    pl.read_csv(IN, has_header=False)
    .with_columns(
        pl.col("column_1")
        .str
        .replace_all(" ", "0")
        .str
        .split("")
        .list
        .to_struct()
    )
    .unnest("column_1")
    .reverse()
    .transpose()
    .rename({"column_0": "ops"})
    # .with_columns(
    #     pl.concat_str(cs.contains("column_"))
    #     .str
    #     .reverse()
    #     .str
    #     .strip_chars("0")
    #     .alias("nums")
    # )
)

shape: (15, 4)
┌─────┬──────────┬──────────┬──────────┐
│ ops ┆ column_1 ┆ column_2 ┆ column_3 │
│ --- ┆ ---      ┆ ---      ┆ ---      │
│ str ┆ str      ┆ str      ┆ str      │
╞═════╪══════════╪══════════╪══════════╡
│ *   ┆ 0        ┆ 0        ┆ 1        │
│ 0   ┆ 0        ┆ 4        ┆ 2        │
│ 0   ┆ 6        ┆ 5        ┆ 3        │
│ 0   ┆ 0        ┆ 0        ┆ 0        │
│ +   ┆ 9        ┆ 6        ┆ 3        │
│ …   ┆ …        ┆ …        ┆ …        │
│ 0   ┆ 5        ┆ 7        ┆ 1        │
│ 0   ┆ 0        ┆ 0        ┆ 0        │
│ +   ┆ 3        ┆ 2        ┆ 6        │
│ 0   ┆ 1        ┆ 3        ┆ 4        │
│ 0   ┆ 4        ┆ 0        ┆ 0        │
└─────┴──────────┴──────────┴──────────┘


4